In [1]:
from astropy.table import Table
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
data_dir = 'C:\\Users\\dubay.11\\OneDrive - The Ohio State University\\Documents\\Docs\\First Year Project\\Data'
data_path = Path(data_dir)

aspcap_file = 'allStarLite-dr17-synspec.fits'
apokasc_file = 'APOKASC_cat_v6.7.2.fits'
starhorse_file = 'APOGEE_DR17_EDR3_STARHORSE_v2.fits'
astroNN_file = 'apogee_astroNN-DR17.fits'
bacchus_file = 'dr17_nc_abund_v1_0.fits'
apok2_file = 'new_apo_k2_updated.csv'

In [3]:
def decode(df):
    """
    Decode DataFrame with byte strings into ordinary strings.

    """
    str_df = df.select_dtypes([object])
    str_df = str_df.stack().str.decode('utf-8').unstack()
    for col in str_df:
        df[col] = str_df[col]
    return df

In [4]:
# Import APOGEE allStar data
aspcap_table = Table.read(data_path / aspcap_file, format='fits', hdu=1)

# Separate paramflags into individual columns
for i in range(len(aspcap_table['PARAMFLAG'][0])):
    aspcap_table['PARAMFLAG' + str(i)] = aspcap_table['PARAMFLAG'][:,i]

# Filter out multidimensional columns
cols = [name for name in aspcap_table.colnames if len(aspcap_table[name].shape) <= 1]
aspcap_df = decode(aspcap_table[cols].to_pandas())

# Relace NaN
aspcap_df.replace(99.999, np.nan, inplace=True)

print(aspcap_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 733901 entries, 0 to 733900
Columns: 191 entries, APOGEE_ID to PARAMFLAG8
dtypes: float32(128), float64(4), int32(32), int64(14), object(13)
memory usage: 621.5+ MB
None


In [5]:
# Import astroNN data
astroNN_table = Table.read(data_path / astroNN_file, format='fits')
# print(astroNN_table.info)
# Select and rename astroNN columns
cols = ['APOGEE_ID'] + astroNN_table.colnames[5:49] 
cols += ['weighted_dist', 'weighted_dist_error', 'age_lowess_correct', 'age_total_error']
cols += ['galr', 'galphi', 'galz']
# print(cols)
cols_new = ['APOGEE_ID'] + ['ASTRONN_'+name for name in cols[1:45]]
cols_new += ['ASTRONN_DIST', 'ASTRONN_DIST_ERR', 'ASTRONN_AGE', 'ASTRONN_AGE_ERR']
cols_new += ['ASTRONN_'+name.upper() for name in cols[49:]]
# print(cols_new)
# Convert to Pandas
astroNN_df = decode(astroNN_table[cols].to_pandas())
astroNN_df.columns = cols_new
print(astroNN_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 733901 entries, 0 to 733900
Data columns (total 52 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   APOGEE_ID           733901 non-null  object 
 1   ASTRONN_TEFF        703046 non-null  float32
 2   ASTRONN_TEFF_ERR    703046 non-null  float32
 3   ASTRONN_LOGG        703040 non-null  float32
 4   ASTRONN_LOGG_ERR    703040 non-null  float32
 5   ASTRONN_C_H         703037 non-null  float32
 6   ASTRONN_C_H_ERR     703037 non-null  float32
 7   ASTRONN_CI_H        703041 non-null  float32
 8   ASTRONN_CI_H_ERR    703041 non-null  float32
 9   ASTRONN_N_H         703033 non-null  float32
 10  ASTRONN_N_H_ERR     703033 non-null  float32
 11  ASTRONN_O_H         703041 non-null  float32
 12  ASTRONN_O_H_ERR     703041 non-null  float32
 13  ASTRONN_NA_H        703044 non-null  float32
 14  ASTRONN_NA_H_ERR    703044 non-null  float32
 15  ASTRONN_MG_H        703043 non-nul

In [56]:
# Join APOGEE and astroNN tables row by row
cat = aspcap_df.join(astroNN_df.drop('APOGEE_ID', axis=1))
# Drop duplicate APOGEE_IDs, keeping highest signal-to-noise (SNREV)
cat = cat.sort_values(['APOGEE_ID', 'SNREV']).drop_duplicates(subset='APOGEE_ID', keep='last').sort_index()
# Drop calibration field (APOGEE_ID = VESTA)
cat = cat[cat['FIELD'].str.contains('calibration')==False]
cat

,APOGEE_ID,TELESCOPE,FIELD,ALT_ID,RA,DEC,GLON,GLAT,J,J_ERR,...,ASTRONN_CO_H_ERR,ASTRONN_NI_H,ASTRONN_NI_H_ERR,ASTRONN_DIST,ASTRONN_DIST_ERR,ASTRONN_AGE,ASTRONN_AGE_ERR,ASTRONN_GALR,ASTRONN_GALPHI,ASTRONN_GALZ
1,2M00000002+7417074,apo25m,120+12,none,0.000103,74.285408,119.401807,11.767414,8.597,0.039,...,0.057862,-0.150067,0.045998,3213.229333,172.268746,6.521422,3.875223,10.045567,0.275933,0.679172
2,2M00000019-1924498,apo25m,060-75,none,0.000832,-19.413851,63.394122,-75.906397,11.074,0.022,...,0.557255,-0.173068,0.067512,509.454764,42.044193,7.751394,1.930535,8.071605,0.013698,-0.471578
4,2M00000032+5737103,apo25m,N7789,none,0.001335,57.619530,116.065371,-4.564768,10.905,0.023,...,0.721272,-0.205134,0.066278,760.736805,5.485313,1.375945,1.130874,8.485873,0.080387,-0.038912
5,2M00000035-7323394,lco25m,SMC12,none,0.001467,-73.394287,307.939441,-43.230305,15.008,0.045,...,0.425350,-1.461033,0.177930,35565.198076,12085.880433,4.181324,2.494765,21.644473,-1.930060,-24.172652
6,2M00000068+5710233,apo25m,N7789,none,0.002850,57.173164,115.977154,-5.002392,10.664,0.023,...,0.342029,-0.104546,0.030763,714.259810,8.741206,6.705514,1.256932,8.460990,0.075663,-0.040685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733896,2M23595886-2009435,apo25m,060-75,none,359.995258,-20.162107,60.806239,-76.324013,11.788,0.022,...,0.019497,-0.439836,0.026048,5539.104691,159.614492,4.710081,2.236992,7.586508,0.151306,-5.364002
733897,2M23595886+5726058,apo25m,116-04,none,359.995265,57.434956,116.025359,-4.745011,11.466,0.024,...,0.526269,-0.480090,0.077579,2042.617726,79.705812,1.907065,0.720498,9.199604,0.199738,-0.145538
733898,2M23595921+5609479,apo25m,NGC7789_MGA,none,359.996744,56.163330,115.771964,-5.991363,11.985,0.023,...,0.083360,-0.237972,0.026719,2479.732340,84.543252,2.331178,0.763392,9.461940,0.236881,-0.235233
733899,2M23595949-7342592,lco25m,SMC12,none,359.997887,-73.716454,307.821174,-42.919934,15.924,0.094,...,0.057444,-0.308245,0.033238,NaN,NaN,9.686720,0.968808,NaN,NaN,NaN


In [12]:
# Import StarHorse
starhorse_table = Table.read(data_path / starhorse_file, format='fits')
# Convert percentiles to minus and plus errors
cols = starhorse_table.colnames[8:-2]
median_cols = [name for name in cols if '50' in name]
cols_new = []
for col in median_cols:
    param = col[:-2]
    mean_col = 'STARHORSE_' + param.upper()
    starhorse_table[mean_col] = starhorse_table[col]
    merr_col = 'STARHORSE_' + param.upper() + '_MERR'
    starhorse_table[merr_col] = starhorse_table[col] - starhorse_table[param+'16']
    perr_col = 'STARHORSE_' + param.upper() + '_PERR'
    starhorse_table[perr_col] = starhorse_table[param+'84'] - starhorse_table[col]
    cols_new += [mean_col, merr_col, perr_col]
cols_new = ['APOGEE_ID'] + cols_new
# print(cols_new)
starhorse_df = decode(starhorse_table[cols_new].to_pandas()).set_index('APOGEE_ID')
print(starhorse_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 562424 entries, 2M01151487-7321065 to 2M01151475-7338069
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   STARHORSE_MASS       562424 non-null  float32
 1   STARHORSE_MASS_MERR  562424 non-null  float32
 2   STARHORSE_MASS_PERR  562424 non-null  float32
 3   STARHORSE_TEFF       562424 non-null  float32
 4   STARHORSE_TEFF_MERR  562424 non-null  float32
 5   STARHORSE_TEFF_PERR  562424 non-null  float32
 6   STARHORSE_LOGG       562424 non-null  float32
 7   STARHORSE_LOGG_MERR  562424 non-null  float32
 8   STARHORSE_LOGG_PERR  562424 non-null  float32
 9   STARHORSE_MET        562424 non-null  float32
 10  STARHORSE_MET_MERR   562424 non-null  float32
 11  STARHORSE_MET_PERR   562424 non-null  float32
 12  STARHORSE_DIST       562424 non-null  float32
 13  STARHORSE_DIST_MERR  562424 non-null  float32
 14  STARHORSE_DIST_PERR  562424 non-null  float3

In [57]:
# Join StarHorse to ASPCAP and astroNN
cat = cat.join(starhorse_df, on='APOGEE_ID')
cat

,APOGEE_ID,TELESCOPE,FIELD,ALT_ID,RA,DEC,GLON,GLAT,J,J_ERR,...,STARHORSE_LOGG_PERR,STARHORSE_MET,STARHORSE_MET_MERR,STARHORSE_MET_PERR,STARHORSE_DIST,STARHORSE_DIST_MERR,STARHORSE_DIST_PERR,STARHORSE_AV,STARHORSE_AV_MERR,STARHORSE_AV_PERR
1,2M00000002+7417074,apo25m,120+12,none,0.000103,74.285408,119.401807,11.767414,8.597,0.039,...,0.000000,-0.100944,0.040165,0.000000,2.741230,0.158694,1.271887,1.395599,1.021982,0.046193
2,2M00000019-1924498,apo25m,060-75,none,0.000832,-19.413851,63.394122,-75.906397,11.074,0.022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2M00000032+5737103,apo25m,N7789,none,0.001335,57.619530,116.065371,-4.564768,10.905,0.023,...,0.015925,-0.199395,0.003737,0.021957,0.761855,0.006993,0.010290,0.973719,0.289089,0.129633
5,2M00000035-7323394,lco25m,SMC12,none,0.001467,-73.394287,307.939441,-43.230305,15.008,0.045,...,0.040237,-1.179234,0.030066,0.040517,36.690964,2.028961,1.953796,0.204770,0.193289,0.180990
6,2M00000068+5710233,apo25m,N7789,none,0.002850,57.173164,115.977154,-5.002392,10.664,0.023,...,0.005898,-0.116648,0.002178,0.000592,0.704613,0.012847,0.012041,0.356403,0.171618,0.192832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733896,2M23595886-2009435,apo25m,060-75,none,359.995258,-20.162107,60.806239,-76.324013,11.788,0.022,...,0.123049,0.100919,0.153004,0.147465,0.463548,0.072889,0.125641,0.250838,0.206427,0.204752
733897,2M23595886+5726058,apo25m,116-04,none,359.995265,57.434956,116.025359,-4.745011,11.466,0.024,...,0.021355,-0.192967,0.000780,0.018056,2.732305,0.098167,0.645983,2.023269,0.394435,0.238733
733898,2M23595921+5609479,apo25m,NGC7789_MGA,none,359.996744,56.163330,115.771964,-5.991363,11.985,0.023,...,0.033506,0.144188,0.020199,0.001790,3.325578,0.285962,0.243858,0.804664,0.224083,0.209489
733899,2M23595949-7342592,lco25m,SMC12,none,359.997887,-73.716454,307.821174,-42.919934,15.924,0.094,...,0.052686,-1.058149,0.026455,0.032227,54.915489,3.214371,2.930302,0.901562,0.214434,0.199528


In [21]:
# Import APOKASC2
apokasc_table = Table.read(data_path / apokasc_file, format='fits')
# Select columns to keep
cols = ['KEPLER_INT', '2MASS_ID'] + [name for name in apokasc_table.colnames if 'APOKASC2_' in name]
# print(apokasc_cols)
apokasc_df = decode(apokasc_table[cols].to_pandas())
# Rename columns with APOKASC2 prefix
apokasc_df.rename(columns={'2MASS_ID': 'APOGEE_ID'}, inplace=True)
apokasc_df.set_index('APOGEE_ID', inplace=True)
# Replace NaN values
apokasc_df.replace([np.inf, -np.inf, -9999., -9999.99, -999., -999.99, '-9999'], np.nan, inplace=True)
apokasc_df

,KEPLER_INT,APOKASC2_NU_MAX,APOKASC2_NU_MAX_ERR,APOKASC2_DELTA_NU,APOKASC2_DELTA_NU_ERR,APOKASC2_EVSTATES,APOKASC2_F_DELTA_NU,APOKASC2_F_DELTA_NU_ERR,APOKASC2_MASS,APOKASC2_MASS_RANERR,...,APOKASC2_LOGG_SYSERR,APOKASC2_DENSITY,APOKASC2_DENSITY_RANDERR,APOKASC2_DENSITY_SYSERR,APOKASC2_AGE,APOKASC2_AGE_PERR,APOKASC2_AGE_MERR,APOKASC2_AV,APOKASC2_AV_ERR,APOKASC2_NOTE
APOGEE_ID,,,,,,,,,,,,,,,,,,,,,
2M19051857+4654038,10000151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2M19052050+4655041,10000162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2M19052775+4658526,10000198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2M19052985+4654372,10000207,94.584999,0.851265,8.838,0.035352,RGB,1.0296,0.0005,1.041,0.040599,...,0.020167,0.00639,0.000051,0.000045,8.413952,1.291148,1.085706,0.053,0.075,...
2M19061343+4657264,10000490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2M18553632+4659372,9995937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2M18554840+4657318,9996021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2M18563273+4658576,9996342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Join APOKASC2 to other catalogues
cat = cat.join(apokasc_df, on='APOGEE_ID')
cat

,APOGEE_ID,TELESCOPE,FIELD,ALT_ID,RA,DEC,GLON,GLAT,J,J_ERR,...,APOKASC2_LOGG_SYSERR,APOKASC2_DENSITY,APOKASC2_DENSITY_RANDERR,APOKASC2_DENSITY_SYSERR,APOKASC2_AGE,APOKASC2_AGE_PERR,APOKASC2_AGE_MERR,APOKASC2_AV,APOKASC2_AV_ERR,APOKASC2_NOTE
1,2M00000002+7417074,apo25m,120+12,none,0.000103,74.285408,119.401807,11.767414,8.597,0.039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2M00000019-1924498,apo25m,060-75,none,0.000832,-19.413851,63.394122,-75.906397,11.074,0.022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2M00000032+5737103,apo25m,N7789,none,0.001335,57.619530,116.065371,-4.564768,10.905,0.023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2M00000035-7323394,lco25m,SMC12,none,0.001467,-73.394287,307.939441,-43.230305,15.008,0.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2M00000068+5710233,apo25m,N7789,none,0.002850,57.173164,115.977154,-5.002392,10.664,0.023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733896,2M23595886-2009435,apo25m,060-75,none,359.995258,-20.162107,60.806239,-76.324013,11.788,0.022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733897,2M23595886+5726058,apo25m,116-04,none,359.995265,57.434956,116.025359,-4.745011,11.466,0.024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733898,2M23595921+5609479,apo25m,NGC7789_MGA,none,359.996744,56.163330,115.771964,-5.991363,11.985,0.023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733899,2M23595949-7342592,lco25m,SMC12,none,359.997887,-73.716454,307.821174,-42.919934,15.924,0.094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# Import BACCHUS carbon isotope ratios
bacchus_table = Table.read(data_path / bacchus_file, format='fits', hdu=1)
cols = ['APOGEE_ID', 'FIELD', 'SNR', 'C12C13', 'C12C13_ERR_MEAS', 'C12C13_ERR_EMP']
bacchus_df = decode(bacchus_table[cols].to_pandas())
# Drop duplicate APOGEE_IDs, keeping highest SNR
bacchus_df = bacchus_df.sort_values(['APOGEE_ID', 'SNR']).drop_duplicates(subset='APOGEE_ID', keep='last').sort_index()
bacchus_df.set_index(['APOGEE_ID', 'FIELD'], inplace=True)
bacchus_df.drop('SNR', axis=1, inplace=True)
# print(bacchus_df.info())
# print(bacchus_df[bacchus_df.duplicated(subset='APOGEE_ID')])
bacchus_df

,,C12C13,C12C13_ERR_MEAS,C12C13_ERR_EMP
APOGEE_ID,FIELD,,,
19140272-1554055,ruprecht147,NaN,NaN,NaN
2M00000506+5656353,NGC7789_btx,NaN,NaN,NaN
2M00000535+1504343,105-45,NaN,NaN,NaN
2M00000546+6152107,116+00,13.179,1.343354,1.024737
2M00000644+5733530,NGC7789_MGA,NaN,NaN,NaN
...,...,...,...,...
AP18135083-3102184,000-06-C,NaN,NaN,NaN
AP18135859-3104469,000-06-C,NaN,NaN,NaN
AP18140813-3113558,000-06-C,20.094,1.782261,1.722627


In [59]:
# Join BACCHUS to other catalogues
cat = cat.join(bacchus_df, on=['APOGEE_ID', 'FIELD'])
cat

,APOGEE_ID,TELESCOPE,FIELD,ALT_ID,RA,DEC,GLON,GLAT,J,J_ERR,...,APOKASC2_DENSITY_SYSERR,APOKASC2_AGE,APOKASC2_AGE_PERR,APOKASC2_AGE_MERR,APOKASC2_AV,APOKASC2_AV_ERR,APOKASC2_NOTE,C12C13,C12C13_ERR_MEAS,C12C13_ERR_EMP
1,2M00000002+7417074,apo25m,120+12,none,0.000103,74.285408,119.401807,11.767414,8.597,0.039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2M00000019-1924498,apo25m,060-75,none,0.000832,-19.413851,63.394122,-75.906397,11.074,0.022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2M00000032+5737103,apo25m,N7789,none,0.001335,57.619530,116.065371,-4.564768,10.905,0.023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2M00000035-7323394,lco25m,SMC12,none,0.001467,-73.394287,307.939441,-43.230305,15.008,0.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2M00000068+5710233,apo25m,N7789,none,0.002850,57.173164,115.977154,-5.002392,10.664,0.023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733896,2M23595886-2009435,apo25m,060-75,none,359.995258,-20.162107,60.806239,-76.324013,11.788,0.022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733897,2M23595886+5726058,apo25m,116-04,none,359.995265,57.434956,116.025359,-4.745011,11.466,0.024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733898,2M23595921+5609479,apo25m,NGC7789_MGA,none,359.996744,56.163330,115.771964,-5.991363,11.985,0.023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733899,2M23595949-7342592,lco25m,SMC12,none,359.997887,-73.716454,307.821174,-42.919934,15.924,0.094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Import APO-K2 catalogue
cols = ['APOGEE_ID', 'new_mass', 'new_mass_err', 'new_radius', 'new_radius_err', 'SNREV']
apok2_df = pd.read_csv(data_path / apok2_file, usecols=cols)
apok2_df.rename(columns={'new_mass': 'APOK2_MASS', 'new_mass_err': 'APOK2_MASS_ERR', 
                         'new_radius': 'APOK2_RADIUS', 'new_radius_err': 'APOK2_RADIUS_ERR'},
                inplace=True)
# Drop duplicate APOGEE_IDs, keeping highest SNREV
apok2_df = apok2_df.sort_values(['APOGEE_ID', 'SNREV']).drop_duplicates(subset='APOGEE_ID', keep='last').sort_index()
apok2_df = apok2_df.drop('SNREV', axis=1).set_index('APOGEE_ID')
print(apok2_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 7776 entries, 2M13305142-0302596 to 2M12231177+0053258
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   APOK2_MASS        7776 non-null   float64
 1   APOK2_MASS_ERR    7776 non-null   float64
 2   APOK2_RADIUS      7776 non-null   float64
 3   APOK2_RADIUS_ERR  7776 non-null   float64
dtypes: float64(4)
memory usage: 303.8+ KB
None


In [60]:
# Join APO-K2 to other catalogues
cat = cat.join(apok2_df, on='APOGEE_ID')
cat

,APOGEE_ID,TELESCOPE,FIELD,ALT_ID,RA,DEC,GLON,GLAT,J,J_ERR,...,APOKASC2_AV,APOKASC2_AV_ERR,APOKASC2_NOTE,C12C13,C12C13_ERR_MEAS,C12C13_ERR_EMP,APOK2_MASS,APOK2_MASS_ERR,APOK2_RADIUS,APOK2_RADIUS_ERR
1,2M00000002+7417074,apo25m,120+12,none,0.000103,74.285408,119.401807,11.767414,8.597,0.039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2M00000019-1924498,apo25m,060-75,none,0.000832,-19.413851,63.394122,-75.906397,11.074,0.022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2M00000032+5737103,apo25m,N7789,none,0.001335,57.619530,116.065371,-4.564768,10.905,0.023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2M00000035-7323394,lco25m,SMC12,none,0.001467,-73.394287,307.939441,-43.230305,15.008,0.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2M00000068+5710233,apo25m,N7789,none,0.002850,57.173164,115.977154,-5.002392,10.664,0.023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733896,2M23595886-2009435,apo25m,060-75,none,359.995258,-20.162107,60.806239,-76.324013,11.788,0.022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733897,2M23595886+5726058,apo25m,116-04,none,359.995265,57.434956,116.025359,-4.745011,11.466,0.024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733898,2M23595921+5609479,apo25m,NGC7789_MGA,none,359.996744,56.163330,115.771964,-5.991363,11.985,0.023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733899,2M23595949-7342592,lco25m,SMC12,none,359.997887,-73.716454,307.821174,-42.919934,15.924,0.094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
